In [1]:
import jsonlines
import pandas as pd
import cufflinks as cf
import subprocess
import json
import os
cf.go_offline()

In [21]:
project="commons-csv.xml.json"

In [22]:
projects_dir="/Users/kuznetsov/work/workspace/immortals/projects/"
rscript="predict_population_size.R"

In [23]:
project_file=os.path.join(projects_dir, project)
p_name=os.path.splitext(project)[0]
table_file=os.path.join(projects_dir,p_name+'.csv')
res_file=os.path.join(projects_dir,p_name+'.estimate.json')

In [24]:
killed_mutants=list()
survived=list()
with jsonlines.open(project_file) as reader:
    for idx,obj in enumerate(reader):
        if obj['status']=='KILLED':
            for t in obj['killing']:
                killed_mutants.append((idx,t))
        if obj['status']=='SURVIVED':
            survived.append(idx)

In [25]:
col_names=['mutant','test']
df=pd.DataFrame(killed_mutants,columns=col_names)

In [43]:
display(df)

,mutant,test
0,6925,testConvertJavaUtil(org.apache.commons.bcel6.u...
1,6926,testConvertJavaUtil(org.apache.commons.bcel6.u...
2,6942,testConvertJavaUtil(org.apache.commons.bcel6.u...
3,6961,testConvertJavaUtil(org.apache.commons.bcel6.u...
4,7215,testSignatureToStringWithGenerics(org.apache.c...
5,7216,testSignatureToStringWithGenerics(org.apache.c...
6,7230,testSignatureToStringWithGenerics(org.apache.c...
7,7235,testSignatureToStringWithGenerics(org.apache.c...
8,7251,testSignatureToStringWithGenerics(org.apache.c...
9,7260,testSignatureToStringWithGenerics(org.apache.c...


In [27]:
m_table=pd.crosstab(df['mutant'],df['test'])

In [28]:
m_table.to_csv(table_file,header=False)

In [29]:
print(f"Number of killed mutants: {m_table.shape[0]}, survived mutants: {len(survived)}")
print(f"Number of tests: {m_table.shape[1]}")

Number of killed mutants: 1087, survived mutants: 878
Number of tests: 38


In [30]:
rs = m_table.sum(axis=1)

In [34]:
rs.iplot(kind='histogram', bins=m_table.shape[1])

In [35]:
args=['Rscript',
      rscript,
     '-a',
     '-m',
     table_file,
     '-o',
     res_file]

In [36]:
sr=subprocess.run(args, capture_output=True)

In [39]:
with open(res_file) as fd:
    res=json.load(fd)

In [40]:
res

{'dataset': '/Users/kuznetsov/work/workspace/immortals/projects/commons-bcel.xml.csv',
 'n.mutants': 1087,
 'n.samples': 38,
 'n.cougth': 1087,
 'estimators': [{'estimator': 'chao_lower',
   'value': 2699.0025,
   'ci': '(1311.731541, 12649.916383)'},
  {'estimator': 'bootstrap',
   'value': 1569.4086,
   'ci': '(1217.449314, 1921.367839)'},
  {'estimator': 'sample_coverage',
   'value': 4864.4321,
   'ci': '(3901.158716, 5827.705501)'},
  {'estimator': 'chao_upperbound',
   'value': 2699.0025,
   'ci': '(1087.000000, 1087.000000)'},
  {'estimator': 'chao2_our',
   'value': 2699.0025,
   'ci': '(2374.255969, 3023.749006)'},
  {'estimator': 'jack_wiqid',
   'value': 3432.757,
   'ci': '(3031.054138, 3917.464243)'},
  {'estimator': 'jack_our',
   'value': 3417.9872,
   'ci': '(3043.278016, 3792.696449)'}]}